# DisplacementMIP - "building fraction damage" vs. "step function" approach

This notebook illustrates and contrasts two different approaches to model displacement from floods for Somalia (same logic holds for Ethiopia and Sudan). 

First, we demonstrate a "building fraction damage" approach, which calculates the fraction of buildings damaged when exposed to hazard maps of various return periods. Based on this building fraction damage, we then apply masks for different thresholds to calculate the displaced population within these buildings. 

The second approach involves applying damage thresholds directly to the impact functions, resulting in step functions. These step functions are then used to calculate the displaced population.

In [1]:
import copy
import numpy as np
import os
from pathlib import Path
import pandas as pd
import sys

from climada.hazard import Hazard
from climada.entity.exposures import Exposures
from climada.engine import ImpactCalc

os.chdir('/Users/simonameiler/Documents/WCR/Displacement/global-displacement-risk')
import exposure
import vulnerability

In [2]:
# Constants
PATH_FL_HAZARD = Path('/Users/simonameiler/Documents/WCR/Displacement/Somalia_CIMA_example/Hazard')

DMG_THRESHS = {'low' : 0.3, 'med' : 0.55, 'high': 0.7}

### Load exposure

In [3]:
cntry_iso = 'SOM'
rcp = 'HISTORICAL'

In [13]:
# load bem, make exp (just admin0)
gdf_bem_subcomps = exposure.gdf_from_bem_subcomps(cntry_iso, opt='full')
gdf_bem_subcomps = gdf_bem_subcomps[gdf_bem_subcomps.valhum>1] # filter out rows with basically no population

exp = Exposures(gdf_bem_subcomps.copy())
exp.value_unit = 'building_unit'
exp.gdf['longitude'] = exp.gdf.geometry.x
exp.gdf['latitude'] = exp.gdf.geometry.y
exp.gdf['value'] = 1

In [5]:
exp.gdf.head()

,id_1x,iso3,cpx,sector,se_seismo,valhum,valfis,bd_1_floor,bd_2_floor,bd_3_floor,geometry,longitude,latitude,value
24,194457751,SOM,3,edu_pub,W2,3.642231,0.011027,0.0,0.0,0.0,POINT (43.40417 0.66250),43.404167,0.6625,1
25,194457751,SOM,3,edu_pub,W1,1.557365,0.004715,0.0,0.0,0.0,POINT (43.40417 0.66250),43.404167,0.6625,1
48,194457751,SOM,3,ic_low,W2,15.383508,0.004852,0.0,0.0,0.0,POINT (43.40417 0.66250),43.404167,0.6625,1
49,194457751,SOM,3,ic_low,W1,6.577763,0.002074,0.0,0.0,0.0,POINT (43.40417 0.66250),43.404167,0.6625,1
51,194457751,SOM,3,ic_low,RS2,1.697488,0.000535,0.0,0.0,0.0,POINT (43.40417 0.66250),43.404167,0.6625,1


In [5]:
country = {'ETH': 'Ethiopia',
           'SOM': 'Somalia',
           'SDN': 'Sudan'}

### Load hazard

In [4]:
HAZ_FOLDER = PATH_FL_HAZARD/cntry_iso/rcp
haz_files = np.sort([str(file) for file in HAZ_FOLDER.glob('*.tif')]).tolist()
rp = np.array([int(Path(file).stem[-4:]) for file in haz_files])

In [5]:
HAZ_FOLDER

PosixPath('/Users/simonameiler/Documents/WCR/Displacement/Somalia_CIMA_example/Hazard/SOM/HISTORICAL')

In [6]:
haz_files

['/Users/simonameiler/Documents/WCR/Displacement/Somalia_CIMA_example/Hazard/SOM/HISTORICAL/HM_defended_T0005.tif',
 '/Users/simonameiler/Documents/WCR/Displacement/Somalia_CIMA_example/Hazard/SOM/HISTORICAL/HM_defended_T0010.tif',
 '/Users/simonameiler/Documents/WCR/Displacement/Somalia_CIMA_example/Hazard/SOM/HISTORICAL/HM_defended_T0025.tif',
 '/Users/simonameiler/Documents/WCR/Displacement/Somalia_CIMA_example/Hazard/SOM/HISTORICAL/HM_defended_T0050.tif',
 '/Users/simonameiler/Documents/WCR/Displacement/Somalia_CIMA_example/Hazard/SOM/HISTORICAL/HM_defended_T0100.tif',
 '/Users/simonameiler/Documents/WCR/Displacement/Somalia_CIMA_example/Hazard/SOM/HISTORICAL/HM_defended_T0200.tif',
 '/Users/simonameiler/Documents/WCR/Displacement/Somalia_CIMA_example/Hazard/SOM/HISTORICAL/HM_defended_T0500.tif',
 '/Users/simonameiler/Documents/WCR/Displacement/Somalia_CIMA_example/Hazard/SOM/HISTORICAL/HM_defended_T1000.tif']

In [9]:
haz = Hazard.from_raster(
haz_type='FL', files_intensity=haz_files, src_crs='WGS84',
attrs={'unit': 'm', 'event_id': np.arange(len(haz_files)), 'frequency':1/rp})

In [10]:
rps = 1/haz.frequency
#rps = np.sort(rps)
rps

array([   2.,    5.,   10.,   25.,   50.,  100.,  250.,  500., 1000.])

In [12]:
rp.tolist()

[2, 5, 10, 25, 50, 100, 250, 500, 1000]

## "Building fraction damage approach"

In [12]:
# compute physical impact (building fraction damage) and save impact matrix for future postproc

# scenario 1: capra/cima impfs
dict_imp_bldg = {}
exp.gdf['impf_FL'] = exp.gdf['se_seismo'].map(vulnerability.DICT_PAGER_FLIMPF_CIMA)
dict_imp_bldg['cima'] = ImpactCalc(exp, vulnerability.IMPF_SET_FL_CIMA, haz).impact(save_mat=True)

# scenario 2: ivm impfs
#exp.gdf['impf_FL'] = exp.gdf['se_seismo'].map(vulnerability.DICT_PAGER_FLIMPF_IVM)
#dict_imp_bldg['ivm'] = ImpactCalc(exp, vulnerability.IMPF_SET_FL_IVM, haz).impact(save_mat=True)

**Show one example**

In [13]:
# let's take cima and the low threshold (0.3 - low) and create sparse boolean impact matrix
imp_map_cima_low = dict_imp_bldg['cima'].imp_mat > DMG_THRESHS['low']
imp_map_cima_low

<9x522899 sparse matrix of type '<class 'numpy.bool_'>'
	with 428791 stored elements in Compressed Sparse Row format>

In [14]:
scen_name = 'cima_low'

# Aggregate the sparse boolean impact matrix (displacement True/False) into the full exposure format per RP.
full_bool = imp_map_cima_low.toarray()

# Ensure the boolean matrix and the exposure dataframe have compatible shapes
assert full_bool.shape[1] == len(exp.gdf), "Shape mismatch between impact matrix and exposure data"

# Calculate impacts (multiply boolean matrix with population values) for each return period and save to exposure GeoDataFrame
for idx, rp in enumerate(rps):
    exp.gdf[f'imp_rp_{rp}_{scen_name}'] = full_bool[idx, :].astype(int) * exp.gdf['valhum']

In [15]:
exp.gdf.head()

,id_1x,iso3,cpx,sector,se_seismo,valhum,valfis,bd_1_floor,bd_2_floor,bd_3_floor,...,centr_FL,imp_rp_2.0_cima_low,imp_rp_5.0_cima_low,imp_rp_10.0_cima_low,imp_rp_25.0_cima_low,imp_rp_50.0_cima_low,imp_rp_100.0_cima_low,imp_rp_250.0_cima_low,imp_rp_500.0_cima_low,imp_rp_1000.0_cima_low
24,194457751,SOM,3,edu_pub,W2,3.642231,0.011027,0.0,0.0,0.0,...,171371565,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25,194457751,SOM,3,edu_pub,W1,1.557365,0.004715,0.0,0.0,0.0,...,171371565,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48,194457751,SOM,3,ic_low,W2,15.383508,0.004852,0.0,0.0,0.0,...,171371565,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49,194457751,SOM,3,ic_low,W1,6.577763,0.002074,0.0,0.0,0.0,...,171371565,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51,194457751,SOM,3,ic_low,RS2,1.697488,0.000535,0.0,0.0,0.0,...,171371565,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# Sum over impacts for each return period
imp_cima_low = exp.gdf.sum()[[f'imp_rp_{rp}_{scen_name}' for rp in rps]]

/var/folders/7j/w3ygw9_d0wb9dn7w0f21n_980000gn/T/ipykernel_93608/1062569006.py:2: FutureWarning: The default value of numeric_only in GeoDataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  imp_cima_low = exp.gdf.sum()[[f'imp_rp_{rp}_{scen_name}' for rp in rps]]


In [17]:
imp_cima_low

imp_rp_2.0_cima_low                  0.0
imp_rp_5.0_cima_low        986280.130932
imp_rp_10.0_cima_low       1170223.91625
imp_rp_25.0_cima_low      1681158.194824
imp_rp_50.0_cima_low      2221687.791797
imp_rp_100.0_cima_low     2372456.857963
imp_rp_250.0_cima_low     2710151.535252
imp_rp_500.0_cima_low     2950019.110034
imp_rp_1000.0_cima_low    3152505.374383
dtype: object

In [18]:
# Compute average annually expected displacement, as sum(displacement(rp) / rp) for all rp.
imp_cima_low[f'aed_{scen_name}'] = 0
for rp in rps:
    imp_cima_low[f'aed_{scen_name}'] += imp_cima_low[f'imp_rp_{rp}_{scen_name}'] / rp
    
imp_cima_low

imp_rp_2.0_cima_low                  0.0
imp_rp_5.0_cima_low        986280.130932
imp_rp_10.0_cima_low       1170223.91625
imp_rp_25.0_cima_low      1681158.194824
imp_rp_50.0_cima_low      2221687.791797
imp_rp_100.0_cima_low     2372456.857963
imp_rp_250.0_cima_low     2710151.535252
imp_rp_500.0_cima_low     2950019.110034
imp_rp_1000.0_cima_low    3152505.374383
aed_cima_low               469576.219755
dtype: object

## "Step function approach"

In [20]:
# for this approach, we set use the population count as key value in the exposure for the impact calculation
exp.gdf.rename({'value': 'valfrac'}, axis=1, inplace=True)
exp.gdf.rename({'valhum': 'value'}, axis=1, inplace=True)
exp.gdf.head()

,id_1x,iso3,cpx,sector,se_seismo,value,valfis,bd_1_floor,bd_2_floor,bd_3_floor,...,centr_FL,imp_rp_2.0_cima_low,imp_rp_5.0_cima_low,imp_rp_10.0_cima_low,imp_rp_25.0_cima_low,imp_rp_50.0_cima_low,imp_rp_100.0_cima_low,imp_rp_250.0_cima_low,imp_rp_500.0_cima_low,imp_rp_1000.0_cima_low
24,194457751,SOM,3,edu_pub,W2,3.642231,0.011027,0.0,0.0,0.0,...,171371565,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25,194457751,SOM,3,edu_pub,W1,1.557365,0.004715,0.0,0.0,0.0,...,171371565,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48,194457751,SOM,3,ic_low,W2,15.383508,0.004852,0.0,0.0,0.0,...,171371565,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49,194457751,SOM,3,ic_low,W1,6.577763,0.002074,0.0,0.0,0.0,...,171371565,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51,194457751,SOM,3,ic_low,RS2,1.697488,0.000535,0.0,0.0,0.0,...,171371565,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
# make step functions for given building threshold
impf_set_fl = vulnerability.IMPF_SET_FL_CIMA

In [22]:
from climada.entity import ImpactFunc, ImpactFuncSet
impf_set_step = ImpactFuncSet()

In [23]:
# The threshold of building damage after which all people are displaced. Below, no-one is displaced.
for imp_id in impf_set_fl.get_ids(haz_type='FL'):
    impf_set_fl.get_func(fun_id=imp_id)
    y = impf_set_fl.get_func(fun_id=imp_id)[0].intensity
    x = impf_set_fl.get_func(fun_id=imp_id)[0].mdd
    flood_thres = np.interp(DMG_THRESHS['low'], x, y)
    print('ID: '+str(imp_id)+' - threshold stepfunction: '+str(flood_thres))
    impf_set_step.append(
                ImpactFunc.from_step_impf(
                    intensity=(0,  flood_thres, flood_thres *10),
                    haz_type='FL',
                    impf_id=imp_id,
                    intensity_unit = 'm'
                )
    )

ID: 1 - threshold stepfunction: 1.6501727272727271
ID: 2 - threshold stepfunction: 3.0897
ID: 3 - threshold stepfunction: 5.8621
ID: 4 - threshold stepfunction: 7.25308813559322
ID: 5 - threshold stepfunction: 10.0
ID: 6 - threshold stepfunction: 1.6494416666666667
ID: 7 - threshold stepfunction: 2.8966
ID: 8 - threshold stepfunction: 4.4510233333333336
ID: 9 - threshold stepfunction: 5.833358333333334
ID: 10 - threshold stepfunction: 7.2498097560975605
ID: 11 - threshold stepfunction: 1.0249714285714286
ID: 12 - threshold stepfunction: 0.7434042553191489
ID: 13 - threshold stepfunction: 0.7436553191489361
ID: 14 - threshold stepfunction: 1.0249714285714286
ID: 15 - threshold stepfunction: 1.4075205479452055


In [24]:
impcalc = ImpactCalc(exp, impf_set_step, haz)
impact = impcalc.impact()

In [34]:
impact.aai_agg  

469576.2197553227

In [33]:
impact.at_event

array([      0.        ,  986280.1309319 , 1170223.91624959,
       1681158.19482406, 2221687.79179678, 2372456.8579626 ,
       2710151.53525227, 2950019.11003364, 3152505.37438334])